# Install dependecies and define helper functions

In [1]:
!pip install transformers
!pip install biopython

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 10.6 MB/s eta 0:00:00


In [2]:
import torch
from torch import nn
import transformers
from transformers import BertTokenizer, BertForTokenClassification
import numpy as np
from Bio import SeqIO
from io import StringIO, BytesIO
from google.colab import drive, files
from tqdm import tqdm
import pickle
import scipy
from scipy import ndimage

In [3]:
def seq2kmer(seq, k):
    kmer = [seq[x:x+k] for x in range(len(seq)+1-k)]
    return kmer

def split_seq(seq, length = 512, pad = 16):
    res = []
    for st in range(0, len(seq), length - pad):
        end = min(st+512, len(seq))
        res.append(seq[st:end])
    return res

def stitch_np_seq(np_seqs, pad = 16):
    res = np.array([])
    for seq in np_seqs:
        res = res[:-pad]
        res = np.concatenate([res,seq])
    return res

# Select model and parameters

In [4]:
model = 'HG kouzine' #@param ["HG chipseq", "HG kouzine", "MM chipseq", "MM kouzine"]
model_confidence_threshold = 0.5 #@param {type:"number"}
minimum_sequence_length = 10 #@param {type:"integer"}

In [5]:
if model == 'HG chipseq':
    model_id = '1VAsp8I904y_J0PUhAQqpSlCn1IqfG0FB'
elif model == 'HG kouzine':
    model_id = '1dAeAt5Gu2cadwDhbc7OnenUgDLHlUvkx'
elif model == 'MM curax':
    model_id = '1W6GEgHNoitlB-xXJbLJ_jDW4BF35W1Sd'
elif model == 'MM kouzine':
    model_id = '1dXpQFmheClKXIEoqcZ7kgCwx6hzVCv3H'


In [6]:
!gdown $model_id
!gdown 10sF8Ywktd96HqAL0CwvlZZUUGj05CGk5
!gdown 16bT7HDv71aRwyh3gBUbKwign1mtyLD2d
!gdown 1EE9goZ2JRSD8UTx501q71lGCk-CK3kqG
!gdown 1gZZdtAoDnDiLQqjQfGyuwt268Pe5sXW0


!mkdir 6-new-12w-0
!mv pytorch_model.bin 6-new-12w-0/
!mv config.json 6-new-12w-0/
!mv special_tokens_map.json 6-new-12w-0/
!mv tokenizer_config.json 6-new-12w-0/
!mv vocab.txt 6-new-12w-0/

Downloading...
From (original): https://drive.google.com/uc?id=1dAeAt5Gu2cadwDhbc7OnenUgDLHlUvkx
From (redirected): https://drive.google.com/uc?id=1dAeAt5Gu2cadwDhbc7OnenUgDLHlUvkx&confirm=t&uuid=9aac0e47-5e01-43fa-8b9b-63337f904683
To: /content/pytorch_model.bin
100% 354M/354M [00:07<00:00, 47.0MB/s]
Downloading...
From: https://drive.google.com/uc?id=10sF8Ywktd96HqAL0CwvlZZUUGj05CGk5
To: /content/config.json
100% 634/634 [00:00<00:00, 2.85MB/s]
Downloading...
From: https://drive.google.com/uc?id=16bT7HDv71aRwyh3gBUbKwign1mtyLD2d
To: /content/special_tokens_map.json
100% 112/112 [00:00<00:00, 431kB/s]
Downloading...
From: https://drive.google.com/uc?id=1EE9goZ2JRSD8UTx501q71lGCk-CK3kqG
To: /content/tokenizer_config.json
100% 40.0/40.0 [00:00<00:00, 167kB/s]
Downloading...
From: https://drive.google.com/uc?id=1gZZdtAoDnDiLQqjQfGyuwt268Pe5sXW0
To: /content/vocab.txt
100% 28.7k/28.7k [00:00<00:00, 55.5MB/s]


In [7]:
tokenizer = BertTokenizer.from_pretrained('6-new-12w-0/')
model = BertForTokenClassification.from_pretrained('6-new-12w-0/')
model.cuda()

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(4101, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

# Upload fasta files for prediction

подгрузка организма

In [8]:
!wget -O 'genome.fna.gz' ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/313/135/GCF_000313135.1_Acastellanii.strNEFF_v1/GCF_000313135.1_Acastellanii.strNEFF_v1_genomic.fna.gz

--2024-06-16 19:35:08--  http://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/313/135/GCF_000313135.1_Acastellanii.strNEFF_v1/GCF_000313135.1_Acastellanii.strNEFF_v1_genomic.fna.gz
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 130.14.250.10, 130.14.250.12, 2607:f220:41e:250::12, ...
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|130.14.250.10|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/313/135/GCF_000313135.1_Acastellanii.strNEFF_v1/GCF_000313135.1_Acastellanii.strNEFF_v1_genomic.fna.gz [following]
--2024-06-16 19:35:08--  https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/313/135/GCF_000313135.1_Acastellanii.strNEFF_v1/GCF_000313135.1_Acastellanii.strNEFF_v1_genomic.fna.gz
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|130.14.250.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12659530 (12M) [application/x-gzip]
Saving to: ‘genome.fna.gz

In [9]:
!gzip -rd 'genome.fna.gz'

# Predict and save raw outputs

In [10]:
out = []
input_file ='genome.fna'
result_dict = {}

for seq_record in SeqIO.parse(input_file, 'fasta'):
    kmer_seq = seq2kmer(str(seq_record.seq).upper(), 6)
    seq_pieces = split_seq(kmer_seq)
    print(seq_record.name)
    with torch.no_grad():
        preds = []
        for seq_piece in tqdm(seq_pieces):
            input_ids = torch.LongTensor(tokenizer.encode(' '.join(seq_piece), add_special_tokens=False))
            outputs = torch.softmax(model(input_ids.cuda().unsqueeze(0))[-1],axis = -1)[0,:,1]
            preds.append(outputs.cpu().numpy())
    result_dict[seq_record.name] = stitch_np_seq(preds)


    labeled, max_label = scipy.ndimage.label(result_dict[seq_record.name]>model_confidence_threshold)
    for label in range(1, max_label+1):
        candidate = np.where(labeled == label)[0]
        candidate_length = candidate.shape[0]
        if candidate_length>minimum_sequence_length:
            print(candidate[0], candidate[-1])
            out.append((seq_record.name, candidate[0], candidate[-1]))

NW_004457733.1


100%|██████████| 101/101 [00:07<00:00, 14.02it/s]


48 59
1342 1355
1451 1463
1553 1574
2293 2314
2580 2595
2713 2728
4490 4504
5068 5084
6181 6200
7564 7578
10909 10927
11699 11717
12478 12491
12822 12838
13080 13125
13129 13142
13350 13367
13437 13460
13576 13618
14540 14552
15136 15150
15157 15171
15669 15704
16011 16027
16125 16149
16158 16174
16441 16455
16720 16733
16882 16906
17183 17198
17214 17234
18199 18218
19149 19164
19369 19383
20003 20017
20562 20575
24366 24383
24453 24471
24491 24511
24588 24602
24933 24953
25436 25460
26467 26483
26705 26768
27121 27138
27302 27315
27512 27549
27748 27775
29076 29124
29249 29263
29330 29344
29518 29536
29798 29831
29985 30067
30179 30195
30256 30313
30529 30544
32946 32960
33186 33206
33255 33269
33463 33500
33508 33525
33740 33758
34071 34089
34161 34176
34313 34328
34472 34497
34532 34548
34686 34706
34715 34743
34757 34774
34780 34797
35093 35112
36243 36262
36306 36327
37262 37284
37407 37428
37450 37527
38174 38191
38320 38332
38460 38475
38642 38669
39084 39102
39115 39135
39371 

100%|██████████| 141/141 [00:04<00:00, 28.39it/s]


5008 5036
5395 5408
5534 5547
5820 5833
6029 6045
6332 6351
6464 6476
6506 6524
6568 6584
7184 7195
18448 18462
18500 18514
18616 18629
19325 19338
19939 19956
20257 20270
20427 20450
20850 20864
21435 21454
21709 21723
22615 22631
23040 23051
24929 24941
25038 25049
25387 25414
25610 25628
26824 26840
27034 27054
27693 27703
28081 28095
28224 28242
28330 28344
28714 28729
28838 28855
29163 29179
29634 29648
29836 29848
30245 30255
30325 30341
30445 30460
30732 30747
31134 31152
31875 31896
33922 33940
35074 35093
35247 35261
35276 35293
35878 35890
36398 36412
36429 36445
36605 36620
36709 36721
37210 37226
37320 37339
37425 37447
37467 37511
37667 37692
37709 37727
38042 38095
38331 38350
38528 38544
38935 38946
39237 39265
39789 39806
39928 39942
39974 39990
40140 40160
40384 40412
40512 40531
40639 40652
41041 41052
41248 41263
42306 42322
42384 42398
42495 42512
43318 43336
44058 44077
44088 44124
44315 44332
44495 44515
44792 44808
44954 44965
45015 45032
45139 45168
45417 45432


...

221 233
397 424
2845 2860
3058 3073
8962 8984
9114 9142
9523 9540
9657 9676
9790 9814
9839 9854
10093 10109
10163 10175
10519 10531
11652 11682
11822 11841
11891 11903
12128 12148
12448 12463
12987 12999
13284 13297
13913 13931
14295 14310
15299 15315
15430 15442
15707 15738
15994 16008
16070 16087
16355 16367
16458 16474
16650 16667
16907 16945
17454 17472
17932 17944
18078 18099
18302 18318
18656 18671
18984 19001
19025 19042
19505 19530
20023 20041
20066 20094
21647 21666
21690 21720
21979 21993
22166 22193
22426 22444
22599 22616
22773 22815
23189 23204
23513 23529
23723 23736
24327 24342
24359 24377
24465 24480
25843 25861
26080 26095
26160 26173
26564 26602
26845 26856
27007 27022
27024 27062
27312 27328
27557 27572
27757 27771
28333 28346
28438 28450
29177 29189
29337 29351
29523 29541
29557 29579
30196 30214
30493 30505
30517 30550
30989 31005
31155 31167
31173 31183
31501 31523
31539 31560
32032 32048
32058 32093
32241 32253
32343 32355
32613 32635
32812 32849
33415 33432
3364

100%|██████████| 31/31 [00:01<00:00, 26.27it/s]


1054 1072
1181 1201
1304 1320
1850 1865
2695 2707
2745 2758
2783 2798
2841 2856
3107 3119
3459 3469
5821 5831
6710 6730
6956 6971
7440 7455
8714 8728
9011 9021
9890 9911
10421 10435
12277 12294
12316 12336
12475 12495
12914 12931
12992 13011
13228 13243
14018 14031
14165 14180
14292 14330
14517 14555
14714 14737
14901 14915
14943 14957
NW_004457242.1


100%|██████████| 292/292 [00:11<00:00, 25.22it/s]


371 385
558 578
662 675
1994 2014
2243 2262
2630 2650
2920 2936
3319 3353
5207 5223
5361 5382
6748 6761
10448 10465
10998 11013
11045 11059
11069 11113
11260 11275
11291 11340
12697 12712
13031 13071
13141 13153
13256 13268
13694 13711
13766 13788
14082 14109
14125 14156
14398 14420
14597 14607
14671 14685
15135 15156
15222 15239
15299 15313
16150 16164
16424 16448
16628 16644
17473 17495
17716 17744
17952 17963
18442 18460
19397 19410
20304 20320
20526 20546
20610 20624
21516 21532
21990 22003
23763 23792
24250 24263
24971 24997
25629 25647
25695 25711
25792 25802
25831 25848
26252 26270
26791 26806
27025 27046
27296 27309
27351 27368
27540 27558
28086 28100
28334 28349
29549 29562
29884 29921
30682 30699
30713 30726
31345 31355
32172 32190
33140 33157
33643 33657
34118 34138
34547 34579
35832 35846
36333 36353
36761 36836
37133 37147
39277 39289
39522 39592
48972 48986
48989 49006
49008 49066
49490 49506
51605 51619
51707 51726
51825 51839
52737 52751
53419 53436
54324 54339
54452 54

In [11]:
out

[('NW_004457733.1', 48, 59),
 ('NW_004457733.1', 1342, 1355),
 ('NW_004457733.1', 1451, 1463),
 ('NW_004457733.1', 1553, 1574),
 ('NW_004457733.1', 2293, 2314),
 ('NW_004457733.1', 2580, 2595),
 ('NW_004457733.1', 2713, 2728),
 ('NW_004457733.1', 4490, 4504),
 ('NW_004457733.1', 5068, 5084),
 ('NW_004457733.1', 6181, 6200),
 ('NW_004457733.1', 7564, 7578),
 ('NW_004457733.1', 10909, 10927),
 ('NW_004457733.1', 11699, 11717),
 ('NW_004457733.1', 12478, 12491),
 ('NW_004457733.1', 12822, 12838),
 ('NW_004457733.1', 13080, 13125),
 ('NW_004457733.1', 13129, 13142),
 ('NW_004457733.1', 13350, 13367),
 ('NW_004457733.1', 13437, 13460),
 ('NW_004457733.1', 13576, 13618),
 ('NW_004457733.1', 14540, 14552),
 ('NW_004457733.1', 15136, 15150),
 ('NW_004457733.1', 15157, 15171),
 ('NW_004457733.1', 15669, 15704),
 ('NW_004457733.1', 16011, 16027),
 ('NW_004457733.1', 16125, 16149),
 ('NW_004457733.1', 16158, 16174),
 ('NW_004457733.1', 16441, 16455),
 ('NW_004457733.1', 16720, 16733),
 ('NW_00445

In [12]:
with open('./ZDNABERT.bed', 'a') as f:
    for record in out:
        f.write(record[0] + "\t" + str(record[1]) + "\t" + str(record[2]) + "\n")